In [1]:
import ee
import geemap
import pprint
from matplotlib import pyplot as plt
import pandas as pd
import os
import numpy as np

In [2]:
# Initialise (don't need maybe?)
ee.Initialize()
# Configure the pretty printing output & initialize earthengine
pp = pprint.PrettyPrinter(depth=4)

## Parameters (INPUT)

In [3]:
# Parameters
maps_outdir = os.path.join(os.path.expanduser('~'), '../Japan_mapping/results/maps')

# Map type: binary or representing rice types (normal, early, second transplants'
# 'yes' for binary, 'no' for maps with different types
# If not binary, map values are: 1: normal rice; 10: early; 20: second; other values are combinations of those (rice detected in different periods)
binarymap = 'yes'  

In [4]:
Map = geemap.Map(center=[38,138], zoom=6)
Map.add_basemap('HYBRID')
Map

Map(center=[38, 138], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(To…

### Load prefectures shapefile and visualize

In [5]:
# Japan shp is not small, so we ingest it from personal account (otherwise we can upload locally with geemap.shp_to_ee)
japan_shp = 'users/luiscartor/japan_gadm0'
japan = geemap.ee.FeatureCollection(japan_shp)

#Map.addLayer(japan, {}, 'Japan')

point = ee.Geometry.Point([38,138])
Map.addLayer(point, {}, 'Point')

# Load prefecture shapefile (from GEE account)
prefs_shp = 'users/luiscartor/japan_gadm1_noOKINAWA'
prefs = geemap.ee.FeatureCollection(prefs_shp)

Map.addLayer(prefs, {}, 'Prefectures')

### Functions

In [6]:
# Function to add year attribute to image
def addyear(img):
    return img.set('year', ee.Image(img).date().get('year'))

# Function to add Day Of Year attribute to image
def addDOY(img):
    return img.set('DOY', ee.Image(img).date().getRelative('day', 'year'))

# Function to full date
def addDATE(img):
    return ee.Image.constant(img.date().getRelative('day', 'year')).int().updateMask(img.select(0).mask())


# Parameters for masking function (move to preamble)
def landsatmasking(img):

  qa = img.select('pixel_qa')
  # If the cloud bit (5) is set and the cloud confidence (7) is high
  # or the cloud shadow bit is set (3), then it's a bad pixel.
  cloudorsnow = qa.bitwiseAnd(1 << 5). \
    And(qa.bitwiseAnd(1 << 7)). \
    Or(qa.bitwiseAnd(1 << 3)). \
    Or(qa.bitwiseAnd(1 << 4))  
  # Remove edge pixels that don't occur in all bands
  mask2 = img.mask().reduce(ee.Reducer.min())
  return img.updateMask(cloudorsnow.Not()).updateMask(mask2)

# Spectral indices function: adds index as a new band to every image
def addINDICES(img):
    
    ndvi = img.normalizedDifference(['B4', 'B3']).rename('NDVI')
    lswi = img.normalizedDifference(['B4', 'B5']).rename('LSWI')
    ndsi = img.normalizedDifference(['B2', 'B5']).rename('NDSI')    
    # I multiply by scale factor for EVI (DIDNT WORK TO SOLVE HIGH VALUES ISSUE)
    evi = img.expression(
    '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
        'NIR': img.select('B4').multiply(0.0001),
      'RED': img.select('B3').multiply(0.0001),
      'BLUE': img.select('B1').multiply(0.0001)
    }).rename('EVI')
    
    return img.addBands(ndvi).addBands(lswi).addBands(ndsi).addBands(evi)

# Function to add a band of rice classification to each image in the col
# Rule for rice is that LSWI − NDVI > 0 or LSWI − EVI > 0
def addRICE(img):    
    rice = img.expression(
    "(lswi - ndvi > 0) || (lswi - evi > 0) ? 1\
    :2",{'lswi': img.select('LSWI'),'ndvi': img.select('NDVI'),'evi': img.select('EVI')}).rename('RICE')
    
    return img.addBands(rice)

# Apply Masks Function
def landcovermasking(img):
  return img.updateMask(mask1.Not()).updateMask(mask2.Not()).updateMask(mask4.Not())

# Function to create binary layer for rice class
def classornot(img):
    return img.eq(1)

### Load data tables

In [7]:
# List of prefectures transplanting START DOY
transplant_startdates_table = '../Japan_mapping/data/transplanting_startdates.csv'
transplant_startdates = pd.read_csv(transplant_startdates_table, sep=",")
transplant_startdates

,Prefecture,8589,9094,9599,0004,0509,1014,1519
0,Hokkaido,142,142,142,140,140,142,139
1,Aomori,134,134,133,132,135,136,135
2,Iwate,128,130,130,130,130,132,130
3,Miyagi,123,124,123,123,124,125,124
4,Akita,134,133,133,133,133,136,135
5,Yamagata,133,133,132,133,133,135,133
6,Fukushima,129,130,128,128,127,131,129
7,Ibaraki,120,121,120,118,117,119,117
8,Tochigi,124,121,121,120,120,121,122
9,Gunma,158,148,141,141,143,143,142


In [8]:
# List of prefectures transplanting START DOY
transplant_enddates_table = '../Japan_mapping/data/transplanting_enddates.csv'
transplant_enddates = pd.read_csv(transplant_enddates_table, sep=",")
transplant_enddates

,Prefecture,8589,9094,9599,0004,0509,1014,1519
0,Hokkaido,151,151,151,149,150,153,150
1,Aomori,144,143,144,145,146,149,148
2,Iwate,142,143,144,142,145,147,145
3,Miyagi,133,134,135,134,139,143,142
4,Akita,143,144,144,145,145,150,150
5,Yamagata,142,142,142,143,144,147,146
6,Fukushima,143,143,142,142,142,146,146
7,Ibaraki,145,145,142,139,139,145,145
8,Tochigi,158,166,168,164,165,166,166
9,Gunma,179,177,177,177,177,179,178


In [9]:
# Some prefectures have "early" or "second" transplantations
# Here we load the dates of those especial transplantation periods
earlytransplant_startdates_table = '../Japan_mapping/data/earlytransplanting_startdates.csv'
earlytransplant_startdates = pd.read_csv(earlytransplant_startdates_table, sep=",")
earlytransplant_enddates_table = '../Japan_mapping/data/earlytransplanting_enddates.csv'
earlytransplant_enddates = pd.read_csv(earlytransplant_enddates_table, sep=",")

secondtransplant_startdates_table = '../Japan_mapping/data/secondtransplanting_startdates.csv'
secondtransplant_startdates = pd.read_csv(secondtransplant_startdates_table, sep=",")
secondtransplant_enddates_table = '../Japan_mapping/data/secondtransplanting_enddates.csv'
secondtransplant_enddates = pd.read_csv(secondtransplant_enddates_table, sep=",")

print(earlytransplant_startdates)
print(earlytransplant_enddates)
print(secondtransplant_startdates)
print(secondtransplant_enddates)

  Prefecture  8589  9094  9599  0004  0509  1014  1519
0      Kochi   102    97    95    94    94    94    93
1   Kumamoto   101    98   100   100   100   100   100
2   Miyazaki    91    85    82    80    80    80    79
3  Kagoshima    93    86    84    82    85    85    78
  Prefecture  8589  9094  9599  0004  0509  1014  1519
0      Kochi   120   119   117   114   112   112   112
1   Kumamoto   114   112   113   113   113   113   113
2   Miyazaki   105    99    97    95    95    97    96
3  Kagoshima   107   105   102   100   102   105   106
  Prefecture  8589  9094  9599  0004  0509  1014  1519
0      Kochi   218   218   218   218   218   218   218
1  Kagoshima   215   215   215   215   215   215   215
  Prefecture  8589  9094  9599  0004  0509  1014  1519
0      Kochi   228   228   228   228   228   228   228
1  Kagoshima   230   230   230   230   230   230   230


In [10]:
# List of prefectures with DOY per temperature data
# temperature_table = '../Japan_mapping/data/prefecturetempdata.csv'
# tempdata = pd.read_csv(temperature_table, sep=",")

temperature_table = '../Japan_mapping/data/prefectureDOYmintemp.csv'
tempdata = pd.read_csv(temperature_table, sep=",")
tempdata

,Prefecture,Meteostation,1,2,3,4,5,6,7,8,...,356,357,358,359,360,361,362,363,364,365
0,Hokkaido,Sapporo,-5.8,-5.9,-6.0,-6.1,-6.3,-6.4,-6.5,-6.6,...,-5.0,-5.0,-5.1,-5.1,-5.2,-5.3,-5.4,-5.5,-5.6,-5.7
1,Aomori,Aomori,-2.8,-2.9,-3.0,-3.1,-3.2,-3.3,-3.3,-3.4,...,-2.1,-2.2,-2.2,-2.3,-2.3,-2.4,-2.5,-2.5,-2.6,-2.7
2,Iwate,Morioka,-4.2,-4.4,-4.5,-4.6,-4.7,-4.9,-5.0,-5.1,...,-3.2,-3.3,-3.4,-3.4,-3.5,-3.6,-3.7,-3.8,-4.0,-4.1
3,Miyagi,Sendai,-0.7,-0.8,-0.9,-1.0,-1.1,-1.2,-1.2,-1.3,...,0.2,0.2,0.1,0.0,-0.1,-0.2,-0.3,-0.4,-0.5,-0.6
4,Akita,Akita,-1.4,-1.5,-1.6,-1.7,-1.8,-1.9,-2.0,-2.1,...,-0.7,-0.7,-0.8,-0.8,-0.9,-1.0,-1.0,-1.1,-1.2,-1.3
5,Yamagata,Yamagata,-2.3,-2.4,-2.5,-2.6,-2.7,-2.8,-2.9,-3.0,...,-1.4,-1.5,-1.5,-1.6,-1.7,-1.8,-1.9,-2.0,-2.1,-2.2
6,Fukushima,Fukushima,-0.9,-1.0,-1.1,-1.2,-1.2,-1.3,-1.4,-1.4,...,0.0,-0.1,-0.2,-0.3,-0.3,-0.4,-0.5,-0.6,-0.7,-0.8
7,Ibaraki,Mito,-1.6,-1.7,-1.8,-1.8,-1.8,-1.9,-1.9,-2.0,...,-0.7,-0.9,-1.0,-1.1,-1.1,-1.2,-1.3,-1.4,-1.5,-1.6
8,Tochigi,Utsunomiya,-2.1,-2.2,-2.2,-2.3,-2.3,-2.4,-2.4,-2.5,...,-1.3,-1.4,-1.5,-1.6,-1.7,-1.7,-1.8,-1.9,-2.0,-2.1
9,Gunma,Maebashi,-0.1,-0.1,-0.2,-0.3,-0.3,-0.4,-0.4,-0.5,...,0.9,0.8,0.7,0.6,0.5,0.4,0.3,0.2,0.1,0.0


### Create table with DOY with first/last 0/5/10 degrees

In [11]:
# Vectors to store TGS values (first day of certain min temperature)
TGS0S = []; TGS0E = []; TGS5S = []; TGS5E = []; TGS10S = []; TGS10E = [];


# We loop through each prefecture
for i in range(len(tempdata)):
    #print(i)
#i = 0

    # Extracts first DOY with mintemp > 0,5,10
    T0S = next(x for x, val in enumerate(tempdata.iloc[i,2:]) if val > 0)
    T5S = next(x for x, val in enumerate(tempdata.iloc[i,2:]) if val > 5)
    T10S = next(x for x, val in enumerate(tempdata.iloc[i,2:]) if val > 10)

    # Extracts first DOY with mintemp < 0,5,10 after summer (we consider temp values since August (DOY=214))
    # If value does not exist (desired temp falls into next year), we set to last day of year (so to not to use following year image)
    T0E = next((x for x, val in enumerate(tempdata.iloc[i,215:]) if val < 0), 365-214) + 214
    T5E = next((x for x, val in enumerate(tempdata.iloc[i,215:]) if val < 5), 365-214) + 214
    T10E = next((x for x, val in enumerate(tempdata.iloc[i,215:]) if val < 10), 365-214) + 214

    TGS0S.append(T0S); TGS5S.append(T5S); TGS10S.append(T10S); TGS0E.append(T0E); TGS5E.append(T5E); TGS10E.append(T10E); 

tgstable = pd.DataFrame({'Prefecture': tempdata['Prefecture'], 'TGS0S': TGS0S, 'TGS0E': TGS0E, 'TGS5S': TGS5S
                        , 'TGS5E': TGS5E, 'TGS10S': TGS10S, 'TGS10E': TGS10E})
tgstable

,Prefecture,TGS0S,TGS0E,TGS5S,TGS5E,TGS10S,TGS10E
0,Hokkaido,90,327,115,302,148,278
1,Aomori,85,339,113,310,144,283
2,Iwate,90,329,116,299,146,278
3,Miyagi,68,360,99,321,128,295
4,Akita,75,349,104,316,133,289
5,Yamagata,81,344,108,311,135,287
6,Fukushima,68,357,99,319,125,293
7,Ibaraki,62,351,96,322,123,297
8,Tochigi,65,348,95,320,120,296
9,Gunma,53,365,91,327,116,301


In [12]:
# Kagoshima gives wrong values due to the TGS5S (above 5 in January but drops later), so we mannualy edit to real value
tgstable['TGS5S'][45] = 40
#tgstable.loc[tgstable['Prefecture']== 'Kagoshima']['TGS5S'] = 40
tgstable

<ipython-input-12-6fd37eae7e9b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  tgstable['TGS5S'][45] = 40


,Prefecture,TGS0S,TGS0E,TGS5S,TGS5E,TGS10S,TGS10E
0,Hokkaido,90,327,115,302,148,278
1,Aomori,85,339,113,310,144,283
2,Iwate,90,329,116,299,146,278
3,Miyagi,68,360,99,321,128,295
4,Akita,75,349,104,316,133,289
5,Yamagata,81,344,108,311,135,287
6,Fukushima,68,357,99,319,125,293
7,Ibaraki,62,351,96,322,123,297
8,Tochigi,65,348,95,320,120,296
9,Gunma,53,365,91,327,116,301


### Define dataframes and lists used in loop 

In [13]:
# Data frame for storing image availability
availabilitytable = pd.DataFrame([])


# Define periods names: dont' edit (used in dictionary loop later)
periodnames = {'8589','9094','9599','0004','0509','1014','1519'}

### Load and process extra datasets for masks 3 and 4

In [14]:
# JAXA FOREST LAYER NOT TOO GOOD SO WE DON'T USE!
# For mask 3: Forest (based on JAXA map)
# Starts from 2007
#forestdataset = ee.ImageCollection('JAXA/ALOS/PALSAR/YEARLY/FNF') \
#    .filterDate('2017-01-01', '2017-12-31');
# Forest is the fnf = 1
#forest = forestdataset.first().select('fnf').eq(1);


# For mask 4 we need elevation data
# Call SRTM elevation dataset
elevdataset = ee.Image('JAXA/ALOS/AW3D30/V2_2')
elevation = elevdataset.select('AVE_DSM')
# Calculate slope
slope = ee.Terrain.slope(elevation)
# Calculate slope > 3 deg
slopegt3 = slope.gt(3)

# Main routine

## Extract collections

In [15]:
# Landsat 5
L5col = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR') \
    .select('B[1-7]','pixel_qa') \
    .map(addyear) \
    .map(addDOY)
       
# Landsat 7
L7col = ee.ImageCollection('LANDSAT/LE07/C01/T1_SR') \
    .select('B[1-7]','pixel_qa') \
    .map(addyear) \
    .map(addDOY)

# Merge collections
L57col = L5col.merge(L7col)

## 5 Year period collections

In [16]:
# Subset to 5-year periods
L57_8589 = L57col.filterDate('1985-01-01', '1989-12-31')
L57_9094 = L57col.filterDate('1990-01-01', '1994-12-31')
L57_9599 = L57col.filterDate('1995-01-01', '1999-12-31')
L57_0004 = L57col.filterDate('2000-01-01', '2004-12-31')
L57_0509 = L57col.filterDate('2005-01-01', '2009-12-31')
L57_1014 = L57col.filterDate('2010-01-01', '2014-12-31')
L57_1519 = L57col.filterDate('2015-01-01', '2019-12-31')

# Dictionary with periods and collections
periods_dict = {'8589': L57_8589,
              '9094': L57_9094,
              '9599': L57_9599,
              '0004': L57_0004,
              '0509': L57_0509,
              '1014': L57_1014,
              '1519': L57_1519}

#transplant_startdates = transplant_startdates.iloc[:2]
#transplant_startdates

## Main loop

In [19]:
# List for storing national rice maps for each period
periodmapslist = list()

# LOOP FOR PERIODS
for periodcol in periods_dict.values():

    #INDENT from here
    #periodcol = L57_8589

    # List for store prefectural maps for current loop period
    prefmapslist = list()


    # LOOP FOR PREFECTURES
    # Script not working for Okinawa!(no data): Japan shapefile does NOT include Okinawa now
    for prefecture in transplant_startdates['Prefecture']:
    #for prefecture in ['Kagoshima']:
    #for prefecture in transplant_startdates['Prefecture'][~np.isin(np.arange(len(transplant_startdates['Prefecture'])), [27,41,46])]:

        #print("Classifying rice for",prefecture,"for the period",list(periods_dict.keys())[list(periods_dict.values()).index(periodcol)])

        # Extract prefecture polygon
        prefpolygon = prefs \
            .filter(ee.Filter.eq("NAME_1", prefecture)) 
        # Clip collection to prefecture
        prefcol = periodcol.filterBounds(prefpolygon)



        # PRE-PROCESSING
        #print("Cloud making")
        prefcol_masked = prefcol.map(landsatmasking)
        # Add Indices
        #print("Adding indices")
        prefcol_indices = prefcol_masked.map(addINDICES)
        # Subset to spectral indices
        prefcol_ind = prefcol_indices.select('NDVI','LSWI','NDSI','EVI')


    #         # IMAGE AVAILABILITY PER PERIOD
    #         # Obtain number of images per year; avoid same date pixels (different tile but same date), we count disctinct dates
    #         freq = prefcol_ind.map(addDATE).reduce(ee.Reducer.countDistinct())

    #         # Visualize map
    #         #Map2.addLayer(freq, {'bands': ['constant_count'],'min':0,'max':50,'palette': ['00FFFF', '0000FF']}, 
    #         #             'Im. Frequency period: ' + list(periods_dict.keys())[list(periods_dict.values()).index(periodcol)])

    #         # Histogram of number of images per pixel
    #         histogram = freq.reduceRegion(**{
    #         'reducer': ee.Reducer.histogram(),
    #         'geometry': prefpolygon.geometry(),
    #         'scale': 30,
    #         'maxPixels': 1e9
    #         })

    #         # Convert histogram (dictionary type) into list
    #         hist = histogram.getInfo()
    #         hist_list = list(hist.values())

    #         # Plot histogram
    #         plt.figure()  # This is needed to plot multiple figures in plot
    #         plt.bar(hist_list[0]['bucketMeans'], hist_list[0]['histogram'], color='g')
    #         plt.ylabel('Number of pixels')
    #         plt.xlabel('Number of images')
    #         plt.title('# of images per pixel for the period '+ 
    #                list(periods_dict.keys())[list(periods_dict.values()).index(periodcol)]+
    #                 ' for '+ prefecture)
    #         # Save figure
    #         plt.savefig('../Japan_mapping/results/figures/imgavailability_period'+
    #                  list(periods_dict.keys())[list(periods_dict.values()).index(periodcol)]+
    #                  prefecture +'_hist.eps', bbox_inches="tight")


        # EXTRACT FLOODING PERIODS AND ESTIMATE IMAGE AVAILABILITY
        # Flooding start day is considered the first day of transplanting (TS)
        flood_start = int(transplant_startdates[list(periods_dict.keys())[list(periods_dict.values()).index(periodcol)]][transplant_startdates[transplant_startdates['Prefecture']==prefecture].index[0]])
        # Flooding last day is considered the end day of transplanting + 30 days
        flood_end = int(transplant_enddates[list(periods_dict.keys())[list(periods_dict.values()).index(periodcol)]][transplant_enddates[transplant_enddates['Prefecture']==prefecture].index[0]] + 30)

        # List for each period (year period and irrigation period)
        # Subset to flooding period
        prefcol_flood = prefcol_ind.filter(ee.Filter.gte('DOY', flood_start)).filter(ee.Filter.lte('DOY', flood_end))

        # Subset to non-flooding period
        prefcol_noflood1 = prefcol_ind.filter(ee.Filter.lte('DOY', flood_start))
        prefcol_noflood2 = prefcol_ind.filter(ee.Filter.gte('DOY', flood_end))
        prefcol_noflood = prefcol_noflood1.merge(prefcol_noflood2)        
        
        
        # AVAILABILITY ANALYSIS
        
        # Add number of images per season and period into table
        #print("Calculate availability")
        #d = {'Prefecture': [prefecture], 
        #     'Lustrum': [list(periods_dict.keys())[list(periods_dict.values()).index(periodcol)]], 
        #     'AvImFlooding': [prefcol_flood.size().getInfo()], 
        #     'AvImNonFlooding': [prefcol_noflood.size().getInfo()]} 
        #availabilitytable_pref = pd.DataFrame(data=d)

        # Add prefecture to table
        #availabilitytable = availabilitytable.append(availabilitytable_pref)

        #print(availabilitytable_pref)
        #print(availabilitytable)

    #         # Histogram of availability during flooding period
    #         freq = prefcol_flood.reduce(ee.Reducer.countDistinct())
    #         # Histogram of number of images per pixel
    #         histogram = freq.reduceRegion(**{
    #           'reducer': ee.Reducer.histogram(),
    #           'geometry': prefpolygon.geometry(),
    #           'scale': 30,
    #           'maxPixels': 1e9
    #         })

    #         # Convert histogram (dictionary type) into list
    #         hist = histogram.getInfo()
    #         hist_list = list(hist.values())

    #         # Plot histogram
    #         plt.figure()  # This is needed to plot multiple figures in plot
    #         plt.bar(hist_list[0]['bucketMeans'], hist_list[0]['histogram'], color='g')
    #         plt.ylabel('Number of pixels')
    #         plt.xlabel('Number of images')
    #         plt.title('# of images per pixel for flooding period during '+ list(periods_dict.keys())[list(periods_dict.values()).index(periodcol)]+
    #                  ' for ' + prefecture)
    #         # Save figure
    #         plt.savefig('../Japan_mapping/results/figures/floodimageavailability_period'+
    #                     list(periods_dict.keys())[list(periods_dict.values()).index(periodcol)]+
    #                     '_'+prefecture+'_hist.eps', bbox_inches="tight")


        
        # EXTRACT FLOODING PERIODS FOR EARLY AND SECOND TRANSPLANTATIONS (WILL ONLY BE USED BY SOME PREFECTURES)
        # Prefectures with early transplantation periods
        if (prefecture =='Kochi' or prefecture =='Kumamoto' or prefecture =='Miyazaki' or prefecture =='Kagoshima'): 
            earlyflood_start = int(earlytransplant_startdates[list(periods_dict.keys())[list(periods_dict.values()).index(periodcol)]][earlytransplant_startdates[earlytransplant_startdates['Prefecture']==prefecture].index[0]])
            earlyflood_end = int(earlytransplant_enddates[list(periods_dict.keys())[list(periods_dict.values()).index(periodcol)]][earlytransplant_enddates[earlytransplant_enddates['Prefecture']==prefecture].index[0]] + 30)
            earlyprefcol_flood = prefcol_ind.filter(ee.Filter.gte('DOY', earlyflood_start)).filter(ee.Filter.lte('DOY', earlyflood_end))

        # Prefectures with second transplantation periods
        if (prefecture =='Kochi' or prefecture =='Kagoshima'): 
            secondlood_start = int(secondtransplant_startdates[list(periods_dict.keys())[list(periods_dict.values()).index(periodcol)]][secondtransplant_startdates[secondtransplant_startdates['Prefecture']==prefecture].index[0]])
            secondflood_end = int(secondtransplant_enddates[list(periods_dict.keys())[list(periods_dict.values()).index(periodcol)]][secondtransplant_enddates[secondtransplant_enddates['Prefecture']==prefecture].index[0]] + 30)
            secondprefcol_flood = prefcol_ind.filter(ee.Filter.gte('DOY', earlyflood_start)).filter(ee.Filter.lte('DOY', earlyflood_end))
        
        
        # FILTER COLLECTIONS FOR TEMPERATURE PERIODS
        #print("Extracting temperature data")
        # Extracting DOY for temperature data
        TGS0S = int(tgstable['TGS0S'][tgstable[tgstable['Prefecture']==prefecture].index[0]])
        TGS0E = int(tgstable['TGS0E'][tgstable[tgstable['Prefecture']==prefecture].index[0]]) 
        TGS5S = int(tgstable['TGS5S'][tgstable[tgstable['Prefecture']==prefecture].index[0]]) 
        TGS5E = int(tgstable['TGS5E'][tgstable[tgstable['Prefecture']==prefecture].index[0]]) 
        TGS10S = int(tgstable['TGS10S'][tgstable[tgstable['Prefecture']==prefecture].index[0]]) 
        TGS10E = int(tgstable['TGS10E'][tgstable[tgstable['Prefecture']==prefecture].index[0]])


        # Subset to period between first 0 and last 0 min degrees:
        L57_0S0E = prefcol_ind.filter(ee.Filter.gte('DOY', TGS0S)).filter(ee.Filter.lte('DOY', TGS0E))
        # Subset to period between first 5 and last 5 min degrees:
        L57_5S5E = prefcol_ind.filter(ee.Filter.gte('DOY', TGS5S)).filter(ee.Filter.lte('DOY', TGS5E))
        # Subset to period between first 0 and first 10 min degrees:
        L57_0S10S = prefcol_ind.filter(ee.Filter.gte('DOY', TGS0S)).filter(ee.Filter.lte('DOY', TGS10S))
        # Subset to period between first 10 + 40days and last 10 min degrees:
        L57_10S40to5E = prefcol_ind.filter(ee.Filter.gte('DOY', TGS10S+40)).filter(ee.Filter.lte('DOY', TGS10E))
        # Subset to period between last 10 and first 10 (not thermal growing season)
        L57_10E10S1 = prefcol_ind.filter(ee.Filter.lte('DOY', TGS10S))
        L57_10E10S2 = prefcol_ind.filter(ee.Filter.gte('DOY', TGS10E))
        L57_10E10S = L57_10E10S1.merge(L57_10E10S2)

        # Calculate median value for indeces for rice flooding period
        L57_flood_median = prefcol_flood.median()


        # APPLY MASKS (based on Dong 2016, RSE)

        # Mask 1) Sparce vegetation: soil, built-up, water body, low vegetated lands. max EVI (T5S-T5E) < 0.6
        # Max with < 0.6 makes me loose many paddy pixels that are paddy (masked out), so using 0.5 
        # BIGGEST TRADE-OFF IS HERE! FOR <0.4, PADDY OVERESTIMATED, FOR <0.6 PADDY UNDERESTIMATED
        L57_5S5E_median = L57_5S5E.median()
        L57_5S5E_max = L57_5S5E.max()

        mask1 = L57_5S5E_max.select('EVI').lt(0.5)

        # Mask 2) Natural vegetation mask: forests, natural wetlands, grass. Max EVI (T10E-T10S) > 0.4
        L57_10E10S_median = L57_10E10S.median()
        L57_10E10S_max = L57_10E10S.max()

        mask2 = L57_10E10S_median.select('EVI').gt(0.4)

        #SOLVE ISSUE: EVI GIVING VALUES ABOVE 2!

        # Mask 3) Forest mask (JAXA): WE DON'T USE!
        # Create forest mask
        #mask3 = forest

        # MASK 3 NOT WORKING VERY WELL. ALOS MAP GETS A LOT OF FALSE POSITIVE FOREST (REDUCING RICE FIELDS): also data only from 2007

        # Mask 4) Slope
        # Create mask for slope > 3 deg
        mask4 = slopegt3

        # APPLY MASKS    
        prefcol_flood_masked =  prefcol_flood.map(landcovermasking)


        # RICE CLASSIFICATION
        # Method 2 for classifying based on % of images
        # Add rice classification layer
        prefcol_ricecol = prefcol_flood_masked.map(addRICE).select('RICE')

        # Now we apply the % of images threshold to classify as rice
        # Reduce the number of images with classornot pixels
        riceprop = prefcol_ricecol.map(classornot).mean()


        # Select % of image necessary to assessing rice!
        riceclass = riceprop.gte(0.1)
        
        
        # CLASSIFY RICE FOR PREFECTURES WITH EARLY AND SECOND PLANTATIONS
        # Prefectures with early transplantation periods
        if (prefecture =='Kochi' or prefecture =='Kumamoto' or prefecture =='Miyazaki' or prefecture =='Kagoshima'): 
            # Apply masks to colection    
            earlyprefcol_flood_masked =  earlyprefcol_flood.map(landcovermasking)
            # Early rice classification
            earlyprefcol_ricecol = earlyprefcol_flood_masked.map(addRICE).select('RICE')
            earlyriceprop = earlyprefcol_ricecol.map(classornot).mean()
            earlyriceclass = earlyriceprop.gte(0.1)
            # Instead of binary map, we assign a value of 10 to early rice
            earlyriceclass = earlyriceclass.multiply(10)
            
            # Add early rice to rice map
            riceclass = riceclass.add(earlyriceclass)
        
        # Prefectures with second transplantation periods
        if (prefecture =='Kochi' or prefecture =='Kagoshima'): 
            # Apply masks to colection    
            secondprefcol_flood_masked =  secondprefcol_flood.map(landcovermasking)
            # Second rice classification
            secondprefcol_ricecol = secondprefcol_flood_masked.map(addRICE).select('RICE')
            secondriceprop = secondprefcol_ricecol.map(classornot).mean()
            secondriceclass = secondriceprop.gte(0.1)
            # Instead of binary map, we assign a value of 20 to second rice
            secondriceclass = secondriceclass.multiply(20)
            
            # Add second rice to rice map
            riceclass = riceclass.add(secondriceclass)
        
        
        # If parameter binarymap is 'yes', the map is converted into binary rice map
        if (binarymap == 'yes'):
            riceclass = riceclass.gte(0.1)
        
        # MOSAICKING
        # Clip image to prefecture
        riceclass_clipped = riceclass.clip(prefpolygon)

        # Insert map in list
        prefmapslist.append(riceclass_clipped)

        # End of prefectures loop

    # Mosaic all prefectural maps for current loop period
    periodricecol = ee.ImageCollection.fromImages(prefmapslist)
    periodmosaic = periodricecol.mosaic()

    # Store period mosaic in list
    periodmapslist.append(periodmosaic)
    
    # Filtering
    periodmosaic_filtered = periodmosaic.reduceNeighborhood(**{   
      'reducer': ee.Reducer.mode(),   
      'kernel': ee.Kernel.square(1), 
      })


    #     # Save period national mosaic to disk: TOO LARGE FOR ALL JAPAN!
    #     filename = os.path.join(maps_outdir, 'ricemap_'+
    #                            list(periods_dict.keys())[list(periods_dict.values()).index(periodcol)]+'.tif')
    #     geemap.ee_export_image(periodmosaic, filename=filename, region=japan.geometry(), scale=30, file_per_band=False)

    # Save period national mosaic to drive: CHANGE REGION TO "japan"
    filename = 'tambo_'+list(periods_dict.keys())[list(periods_dict.values()).index(periodcol)]
    geemap.ee_export_image_to_drive(periodmosaic, description=filename, folder='GEEexports', region=japan.geometry(), scale=30)       

    
    # Export filtered images
    filenamefiltered = 'tambo_filtered_'+list(periods_dict.keys())[list(periods_dict.values()).index(periodcol)]
    geemap.ee_export_image_to_drive(periodmosaic_filtered, description=filenamefiltered, folder='GEEexports', region=japan.geometry(), scale=30)       



Exporting tambo_8589 ...
Exporting tambo_filtered_8589 ...
Exporting tambo_9094 ...
Exporting tambo_filtered_9094 ...
Exporting tambo_9599 ...
Exporting tambo_filtered_9599 ...
Exporting tambo_0004 ...
Exporting tambo_filtered_0004 ...
Exporting tambo_0509 ...
Exporting tambo_filtered_0509 ...
Exporting tambo_1014 ...
Exporting tambo_filtered_1014 ...
Exporting tambo_1519 ...
Exporting tambo_filtered_1519 ...


In [58]:
#print(availabilitytable)
#print(transplant_startdates['Prefecture'])
#print(transplant_startdates['Prefecture'][:-1])

        

In [114]:
Map2 = geemap.Map(center=[38,138], zoom=6)
Map2.add_basemap('HYBRID')
Map2

Map(center=[38, 138], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(To…

In [116]:
#print(len(periodmapslist))
prefcol = ee.ImageCollection.fromImages(prefmapslist)
#print(prefcol.size().getInfo())


prefpolygon = prefs \
        .filter(ee.Filter.eq("NAME_1", 'Hyogo')) 


Map2.addLayer(prefpolygon, {}, 'Hyogo')

#Map2.addLayer(periodmapslist[0], {}, 'period1')
#Map2.addLayer(periodmapslist[1], {}, 'period2')
#Map2.addLayer(periodmapslist[2], {}, 'period3')
#Map2.addLayer(periodmapslist[3], {}, 'period4')
#Map2.addLayer(periodmapslist[4], {}, 'period5')
#Map2.addLayer(periodmapslist[5], {}, 'period6')
#Map2.addLayer(periodmapslist[6], {}, 'period7')

## Split Maps

In [3]:
Map3 = geemap.Map(center=[38,138], zoom=7)
right_layer = geemap.ee_tile_layer(periodmapslist[0], {}, 'Rice 1985-89')
left_layer = geemap.ee_tile_layer(periodmapslist[6], {}, 'Rice 2015-19')
#Map3.split_map(left_layer, right_layer)
Map3.add_basemap('HYBRID')
Map3.split_map(left_layer='HYBRID', right_layer=right_layer)
Map3

NameError: name 'periodmapslist' is not defined

In [4]:
Map4 = geemap.Map(center=[38,138], zoom=7)
right_layer = geemap.ee_tile_layer(periodmapslist[0], {}, 'Rice 1985-90')
left_layer = geemap.ee_tile_layer(periodmapslist[6], {}, 'Rice 2015-2020')
Map4.split_map(left_layer, right_layer)
Map4

NameError: name 'periodmapslist' is not defined

## Loaded Split Maps

In [ ]:
# Load rice maps
tambo1519_path = 'users/luiscartor/tambo_1519_filtered'
tambo1519 = geemap.ee.Image(tambo1519_path)
tambo1014_path = 'users/luiscartor/tambo_1014_filtered'
tambo1014 = geemap.ee.Image(tambo1014_path)
tambo0509_path = 'users/luiscartor/tambo_0509_filtered'
tambo0509 = geemap.ee.Image(tambo0509_path)
tambo0004_path = 'users/luiscartor/tambo_0004_filtered'
tambo0004 = geemap.ee.Image(tambo0004_path)
tambo9599_path = 'users/luiscartor/tambo_9599_filtered'
tambo9599 = geemap.ee.Image(tambo9599_path)
tambo9094_path = 'users/luiscartor/tambo_9094_filtered'
tambo9094 = geemap.ee.Image(tambo9094_path)
tambo8589_path = 'users/luiscartor/tambo_8589_filtered'
tambo8589 = geemap.ee.Image(tambo8589_path)

# Load JAXA map
lc2015_path = 'users/luiscartor/jaxaLC2015'
#lc2015_path = '/Users/luis/Documents/research/jsps/ricemapping/data/JaxaLandCover/jaxaLC2015.tif'
lc2015 = geemap.ee.Image(lc2015_path)

### Split map: RICE VS GOOGLE EARTH

In [ ]:
# Define palette for Rice maps
ricepalette = [
  'FFFFFF', # no rice
   'cdb33b'# rice
]


Map5 = geemap.Map(center=[38,138], zoom=7)

right_layer = geemap.ee_tile_layer(tambo8589, {'min': 0, 'max': 1, 'palette': ricepalette}, 'Rice 1985-89')
left_layer = geemap.ee_tile_layer(tambo1519, {'min': 0, 'max': 1, 'palette': ricepalette}, 'Rice 2015-19')

Map5.add_basemap('HYBRID')
Map5.split_map(left_layer='HYBRID', right_layer=right_layer)
Map5

### Split map: RICE 85-89 VS RICE 15-19

In [ ]:
Map6 = geemap.Map(center=[38,138], zoom=7)

right_layer = geemap.ee_tile_layer(tambo8589, {'min': 0, 'max': 1, 'palette': ricepalette}, 'Rice 1985-89')
left_layer = geemap.ee_tile_layer(tambo1519, {'min': 0, 'max': 1, 'palette': ricepalette}, 'Rice 2015-19')
#Map3.split_map(left_layer, right_layer)
Map6.add_basemap('HYBRID')
Map6.split_map(left_layer=left_layer, right_layer=right_layer)
Map6

### Split map: RICE 15-19 VS JAXA RICE

In [ ]:
# Define a palette for the 18 distinct land cover classes.
jaxaPalette = [
  'aec3d4', 
  '152106', 
  'cdb33b',  # rice
  '111149',
  'cdb33b', # 
  'cc0013', # 
  '6a2325', # 
  'd7cdcc', # 
  'f7e084', # 
  '6f6f6f'  # 
]

Map7 = geemap.Map(center=[38,138], zoom=7)

right_layer = geemap.ee_tile_layer(tambo8589, {'min': 0, 'max': 1, 'palette': ricepalette}, 'Rice 1985-89')
left_layer = geemap.ee_tile_layer(lc2015, {'min': 0, 'max': 10, 'palette': jaxaPalette}, 'JAXA Rice')

Map7.add_basemap('HYBRID')
Map7.split_map(left_layer=left_layer, right_layer=right_layer)
Map7

In [33]:
# List for storing national rice maps for each period
periodmapslist = list()

# LOOP FOR PERIODS
#for periodcol in periods_dict.values():

#INDENT from here
periodcol = L57_8589

# List for store prefectural maps for current loop period
prefmapslist = list()


# LOOP FOR PREFECTURES
# Script not working for Okinawa!(no data): Japan shapefile does NOT include Okinawa now
#for prefecture in transplant_startdates['Prefecture'][:-1]:
for prefecture in ['Kagoshima']:
#for prefecture in transplant_startdates['Prefecture'][~np.isin(np.arange(len(transplant_startdates['Prefecture'])), [27,41,46])]:

    #print("Classifying rice for",prefecture,"for the period",list(periods_dict.keys())[list(periods_dict.values()).index(periodcol)])

    # Extract prefecture polygon
    prefpolygon = prefs \
        .filter(ee.Filter.eq("NAME_1", prefecture)) 
    # Clip collection to prefecture
    prefcol = periodcol.filterBounds(prefpolygon)



    # PRE-PROCESSING
    #print("Cloud making")
    prefcol_masked = prefcol.map(landsatmasking)
    # Add Indices
    #print("Adding indices")
    prefcol_indices = prefcol_masked.map(addINDICES)
    # Subset to spectral indices
    prefcol_ind = prefcol_indices.select('NDVI','LSWI','NDSI','EVI')


#         # IMAGE AVAILABILITY PER PERIOD
#         # Obtain number of images per year; avoid same date pixels (different tile but same date), we count disctinct dates
#         freq = prefcol_ind.map(addDATE).reduce(ee.Reducer.countDistinct())

#         # Visualize map
#         #Map2.addLayer(freq, {'bands': ['constant_count'],'min':0,'max':50,'palette': ['00FFFF', '0000FF']}, 
#         #             'Im. Frequency period: ' + list(periods_dict.keys())[list(periods_dict.values()).index(periodcol)])

#         # Histogram of number of images per pixel
#         histogram = freq.reduceRegion(**{
#         'reducer': ee.Reducer.histogram(),
#         'geometry': prefpolygon.geometry(),
#         'scale': 30,
#         'maxPixels': 1e9
#         })

#         # Convert histogram (dictionary type) into list
#         hist = histogram.getInfo()
#         hist_list = list(hist.values())

#         # Plot histogram
#         plt.figure()  # This is needed to plot multiple figures in plot
#         plt.bar(hist_list[0]['bucketMeans'], hist_list[0]['histogram'], color='g')
#         plt.ylabel('Number of pixels')
#         plt.xlabel('Number of images')
#         plt.title('# of images per pixel for the period '+ 
#                list(periods_dict.keys())[list(periods_dict.values()).index(periodcol)]+
#                 ' for '+ prefecture)
#         # Save figure
#         plt.savefig('../Japan_mapping/results/figures/imgavailability_period'+
#                  list(periods_dict.keys())[list(periods_dict.values()).index(periodcol)]+
#                  prefecture +'_hist.eps', bbox_inches="tight")


    # EXTRACT FLOODING PERIODS AND ESTIMATE IMAGE AVAILABILITY
    # Flooding start day is considered the first day of transplanting (TS)
    flood_start = int(transplant_startdates[list(periods_dict.keys())[list(periods_dict.values()).index(periodcol)]][transplant_startdates[transplant_startdates['Prefecture']==prefecture].index[0]])
    # Flooding last day is considered the end day of transplanting + 30 days
    flood_end = int(transplant_enddates[list(periods_dict.keys())[list(periods_dict.values()).index(periodcol)]][transplant_enddates[transplant_enddates['Prefecture']==prefecture].index[0]] + 30)

    # List for each period (year period and irrigation period)
    # Subset to flooding period
    prefcol_flood = prefcol_ind.filter(ee.Filter.gte('DOY', flood_start)).filter(ee.Filter.lte('DOY', flood_end))

    # Subset to non-flooding period
    prefcol_noflood1 = prefcol_ind.filter(ee.Filter.lte('DOY', flood_start))
    prefcol_noflood2 = prefcol_ind.filter(ee.Filter.gte('DOY', flood_end))
    prefcol_noflood = prefcol_noflood1.merge(prefcol_noflood2)        


    # AVAILABILITY ANALYSIS

    # Add number of images per season and period into table
    #print("Calculate availability")
    #d = {'Prefecture': [prefecture], 
    #     'Lustrum': [list(periods_dict.keys())[list(periods_dict.values()).index(periodcol)]], 
    #     'AvImFlooding': [prefcol_flood.size().getInfo()], 
    #     'AvImNonFlooding': [prefcol_noflood.size().getInfo()]} 
    #availabilitytable_pref = pd.DataFrame(data=d)

    # Add prefecture to table
    #availabilitytable = availabilitytable.append(availabilitytable_pref)

    #print(availabilitytable_pref)
    #print(availabilitytable)

#         # Histogram of availability during flooding period
#         freq = prefcol_flood.reduce(ee.Reducer.countDistinct())
#         # Histogram of number of images per pixel
#         histogram = freq.reduceRegion(**{
#           'reducer': ee.Reducer.histogram(),
#           'geometry': prefpolygon.geometry(),
#           'scale': 30,
#           'maxPixels': 1e9
#         })

#         # Convert histogram (dictionary type) into list
#         hist = histogram.getInfo()
#         hist_list = list(hist.values())

#         # Plot histogram
#         plt.figure()  # This is needed to plot multiple figures in plot
#         plt.bar(hist_list[0]['bucketMeans'], hist_list[0]['histogram'], color='g')
#         plt.ylabel('Number of pixels')
#         plt.xlabel('Number of images')
#         plt.title('# of images per pixel for flooding period during '+ list(periods_dict.keys())[list(periods_dict.values()).index(periodcol)]+
#                  ' for ' + prefecture)
#         # Save figure
#         plt.savefig('../Japan_mapping/results/figures/floodimageavailability_period'+
#                     list(periods_dict.keys())[list(periods_dict.values()).index(periodcol)]+
#                     '_'+prefecture+'_hist.eps', bbox_inches="tight")



    # EXTRACT FLOODING PERIODS FOR EARLY AND SECOND TRANSPLANTATIONS (WILL ONLY BE USED BY SOME PREFECTURES)
    # Prefectures with early transplantation periods
    if (prefecture =='Kochi' or prefecture =='Kumamoto' or prefecture =='Miyazaki' or prefecture =='Kagoshima'): 
        earlyflood_start = int(earlytransplant_startdates[list(periods_dict.keys())[list(periods_dict.values()).index(periodcol)]][earlytransplant_startdates[earlytransplant_startdates['Prefecture']==prefecture].index[0]])
        earlyflood_end = int(earlytransplant_enddates[list(periods_dict.keys())[list(periods_dict.values()).index(periodcol)]][earlytransplant_enddates[earlytransplant_enddates['Prefecture']==prefecture].index[0]] + 30)
        earlyprefcol_flood = prefcol_ind.filter(ee.Filter.gte('DOY', earlyflood_start)).filter(ee.Filter.lte('DOY', earlyflood_end))

    # Prefectures with second transplantation periods
    if (prefecture =='Kochi' or prefecture =='Kagoshima'): 
        secondlood_start = int(secondtransplant_startdates[list(periods_dict.keys())[list(periods_dict.values()).index(periodcol)]][secondtransplant_startdates[secondtransplant_startdates['Prefecture']==prefecture].index[0]])
        secondflood_end = int(secondtransplant_enddates[list(periods_dict.keys())[list(periods_dict.values()).index(periodcol)]][secondtransplant_enddates[secondtransplant_enddates['Prefecture']==prefecture].index[0]] + 30)
        secondprefcol_flood = prefcol_ind.filter(ee.Filter.gte('DOY', earlyflood_start)).filter(ee.Filter.lte('DOY', earlyflood_end))


    # FILTER COLLECTIONS FOR TEMPERATURE PERIODS
    #print("Extracting temperature data")
    # Extracting DOY for temperature data
    TGS0S = int(tgstable['TGS0S'][tgstable[tgstable['Prefecture']==prefecture].index[0]])
    TGS0E = int(tgstable['TGS0E'][tgstable[tgstable['Prefecture']==prefecture].index[0]]) 
    TGS5S = int(tgstable['TGS5S'][tgstable[tgstable['Prefecture']==prefecture].index[0]]) 
    TGS5E = int(tgstable['TGS5E'][tgstable[tgstable['Prefecture']==prefecture].index[0]]) 
    TGS10S = int(tgstable['TGS10S'][tgstable[tgstable['Prefecture']==prefecture].index[0]]) 
    TGS10E = int(tgstable['TGS10E'][tgstable[tgstable['Prefecture']==prefecture].index[0]])


    # Subset to period between first 0 and last 0 min degrees:
    L57_0S0E = prefcol_ind.filter(ee.Filter.gte('DOY', TGS0S)).filter(ee.Filter.lte('DOY', TGS0E))
    # Subset to period between first 5 and last 5 min degrees:
    L57_5S5E = prefcol_ind.filter(ee.Filter.gte('DOY', TGS5S)).filter(ee.Filter.lte('DOY', TGS5E))
    # Subset to period between first 0 and first 10 min degrees:
    L57_0S10S = prefcol_ind.filter(ee.Filter.gte('DOY', TGS0S)).filter(ee.Filter.lte('DOY', TGS10S))
    # Subset to period between first 10 + 40days and last 10 min degrees:
    L57_10S40to5E = prefcol_ind.filter(ee.Filter.gte('DOY', TGS10S+40)).filter(ee.Filter.lte('DOY', TGS10E))
    # Subset to period between last 10 and first 10 (not thermal growing season)
    L57_10E10S1 = prefcol_ind.filter(ee.Filter.lte('DOY', TGS10S))
    L57_10E10S2 = prefcol_ind.filter(ee.Filter.gte('DOY', TGS10E))
    L57_10E10S = L57_10E10S1.merge(L57_10E10S2)

    # Calculate median value for indeces for rice flooding period
    L57_flood_median = prefcol_flood.median()


    # APPLY MASKS (based on Dong 2016, RSE)

    # Mask 1) Sparce vegetation: soil, built-up, water body, low vegetated lands. max EVI (T5S-T5E) < 0.6
    # Max with < 0.6 makes me loose many paddy pixels that are paddy (masked out), so using 0.5 
    # BIGGEST TRADE-OFF IS HERE! FOR <0.4, PADDY OVERESTIMATED, FOR <0.6 PADDY UNDERESTIMATED
    L57_5S5E_median = L57_5S5E.median()
    L57_5S5E_max = L57_5S5E.max()

    mask1 = L57_5S5E_max.select('EVI').lt(0.5)

    # Mask 2) Natural vegetation mask: forests, natural wetlands, grass. Max EVI (T10E-T10S) > 0.4
    L57_10E10S_median = L57_10E10S.median()
    L57_10E10S_max = L57_10E10S.max()

    mask2 = L57_10E10S_median.select('EVI').gt(0.4)

    #SOLVE ISSUE: EVI GIVING VALUES ABOVE 2!

    # Mask 3) Forest mask (JAXA): WE DON'T USE!
    # Create forest mask
    #mask3 = forest

    # MASK 3 NOT WORKING VERY WELL. ALOS MAP GETS A LOT OF FALSE POSITIVE FOREST (REDUCING RICE FIELDS): also data only from 2007

    # Mask 4) Slope
    # Create mask for slope > 3 deg
    mask4 = slopegt3

    # APPLY MASKS    
    prefcol_flood_masked =  prefcol_flood.map(landcovermasking)


    # RICE CLASSIFICATION
    # Method 2 for classifying based on % of images
    # Add rice classification layer
    prefcol_ricecol = prefcol_flood_masked.map(addRICE).select('RICE')

    # Now we apply the % of images threshold to classify as rice
    # Reduce the number of images with classornot pixels
    riceprop = prefcol_ricecol.map(classornot).mean()


    # Select % of image necessary to assessing rice!
    riceclass = riceprop.gte(0.1)


    # CLASSIFY RICE FOR PREFECTURES WITH EARLY AND SECOND PLANTATIONS
    # Prefectures with early transplantation periods
    if (prefecture =='Kochi' or prefecture =='Kumamoto' or prefecture =='Miyazaki' or prefecture =='Kagoshima'): 
        # Apply masks to colection    
        earlyprefcol_flood_masked =  earlyprefcol_flood.map(landcovermasking)
        # Early rice classification
        earlyprefcol_ricecol = earlyprefcol_flood_masked.map(addRICE).select('RICE')
        earlyriceprop = earlyprefcol_ricecol.map(classornot).mean()
        earlyriceclass = earlyriceprop.gte(0.1)
        # Instead of binary map, we assign a value of 10 to early rice
        earlyriceclass = earlyriceclass.multiply(10)

        # Add early rice to rice map
        riceclass = riceclass.add(earlyriceclass)

    # Prefectures with second transplantation periods
    if (prefecture =='Kochi' or prefecture =='Kagoshima'): 
        # Apply masks to colection    
        secondprefcol_flood_masked =  secondprefcol_flood.map(landcovermasking)
        # Second rice classification
        secondprefcol_ricecol = secondprefcol_flood_masked.map(addRICE).select('RICE')
        secondriceprop = secondprefcol_ricecol.map(classornot).mean()
        secondriceclass = secondriceprop.gte(0.1)
        # Instead of binary map, we assign a value of 20 to second rice
        secondriceclass = secondriceclass.multiply(20)

        # Add second rice to rice map
        riceclass = riceclass.add(secondriceclass)


    # If parameter binarymap is 'yes', the map is converted into binary rice map
    if (binarymap == 'yes'):
        riceclass = riceclass.gte(0.1)

    # MOSAICKING
    # Clip image to prefecture
    riceclass_clipped = riceclass.clip(prefpolygon)

    # Insert map in list
    prefmapslist.append(riceclass_clipped)

    # End of prefectures loop

# Mosaic all prefectural maps for current loop period
periodricecol = ee.ImageCollection.fromImages(prefmapslist)
periodmosaic = periodricecol.mosaic()

# Store period mosaic in list
periodmapslist.append(periodmosaic)

# Filtering
periodmosaic_filtered = periodmosaic.reduceNeighborhood(**{   
  'reducer': ee.Reducer.mode(),   
  'kernel': ee.Kernel.square(1), 
  })


#     # Save period national mosaic to disk: TOO LARGE FOR ALL JAPAN!
#     filename = os.path.join(maps_outdir, 'ricemap_'+
#                            list(periods_dict.keys())[list(periods_dict.values()).index(periodcol)]+'.tif')
#     geemap.ee_export_image(periodmosaic, filename=filename, region=japan.geometry(), scale=30, file_per_band=False)

# Save period national mosaic to drive: CHANGE REGION TO "japan"
filename = 'tamboKagoshima_'+list(periods_dict.keys())[list(periods_dict.values()).index(periodcol)]
geemap.ee_export_image_to_drive(periodmosaic, description=filename, folder='GEEexports', region=japan.geometry(), scale=30)       


# Export filtered images
#filenamefiltered = 'tambo_filtered_'+list(periods_dict.keys())[list(periods_dict.values()).index(periodcol)]
#geemap.ee_export_image_to_drive(periodmosaic_filtered, description=filenamefiltered, folder='GEEexports', region=japan.geometry(), scale=30)       



Exporting tamboKagoshima_8589 ...


In [18]:
transplant_startdates['Prefecture']

0      Hokkaido
1        Aomori
2         Iwate
3        Miyagi
4         Akita
5      Yamagata
6     Fukushima
7       Ibaraki
8       Tochigi
9         Gunma
10      Saitama
11        Chiba
12        Tokyo
13     Kanagawa
14      Niigata
15       Toyama
16     Ishikawa
17        Fukui
18    Yamanashi
19       Nagano
20         Gifu
21     Shizuoka
22        Aichi
23          Mie
24        Shiga
25        Kyoto
26        Osaka
27        Hyogo
28         Nara
29     Wakayama
30      Tottori
31      Shimane
32      Okayama
33    Hiroshima
34    Yamaguchi
35    Tokushima
36       Kagawa
37        Ehime
38        Kochi
39      Fukuoka
40         Saga
41     Nagasaki
42     Kumamoto
43         Oita
44     Miyazaki
45    Kagoshima
Name: Prefecture, dtype: object